In [1]:
import pandas as pd
from IPython.display import display

In [13]:
file_path = "/Users/kumar/fake_reviews_prediction/Datasets/fake_reviews_dataset.csv"  # Example: 'data/reviews.csv'
df = pd.read_csv(file_path)

In [24]:
print("Full Dataset:")
display(df.head())


Full Dataset:


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [25]:
required_columns = {'category', 'rating', 'label', 'text_'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing columns! Expected {required_columns}, found {set(df.columns)}")

In [14]:
category_df = df[['category']].copy()
rating_df = df[['rating']].copy()
label_df = df[['label']].copy()
text_df = df[['text_']].copy()

In [4]:
print("\nText DataFrame:")
display(text_df.head())


Text DataFrame:


,text_
0,"Love this! Well made, sturdy, and very comfor..."
1,"love it, a great upgrade from the original. I..."
2,This pillow saved my back. I love the look and...
3,"Missing information on how to use it, but it i..."
4,Very nice set. Good quality. We have had the s...


In [60]:
import requests
model = "deepseek-r1:1.5b"

In [48]:
# Function to query the local LLM server

def query_llm(prompt: str, model: str = model) -> str: # arguments promt and model
    # Send a request to the local LLM server, stream set to Falsefor single responce
    response = requests.post(
        "http://localhost:11434/api/generate",  
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"]

In [61]:
# Model Response with DeepSeek r1 1.5b
text=str(text_df['text_'].iloc[[0]].values[0])
print("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text))


Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word 
Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty
Original


In [ ]:
model = "llama3.2:1b"

In [59]:
# Model Response with Gemma 3

text=str(text_df['text_'].iloc[[0]].values[0])
print("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text))

Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word 
Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty
Original


In [55]:
model = "qwen2:0.5b"

In [57]:
# Model Response with qwen2:0.5b

text=str(text_df['text_'].iloc[[0]].values[0])
print("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text)
print(query_llm("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text))

Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word 
Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty
Original


In [4]:
import pandas as pd
import requests
from tqdm import tqdm

In [10]:
model = "qwen2:0.5b"

In [18]:
model = "deepseek-r1:1.5b"

In [4]:
model = "gemma3:4b"

In [15]:
model = "llama3.2:1b"

In [16]:

merged_df = pd.concat([text_df.reset_index(drop=True),
                       label_df.reset_index(drop=True)], axis=1)

#merged_df = merged_df.fillna("")

sample_df = merged_df.head(1000).copy()
#sample_df = text_df.head(10).copy()

In [17]:
def query_llm(prompt: str, model: str = model) -> str:
    """Send text to Ollama model and return response."""
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": model, "prompt": str(prompt), "stream": False}
        )
        response.raise_for_status()
        return response.json().get("response", "")
    except Exception as e:
        print(f"Error for prompt: {e}")
        return ""


In [18]:
results = []
print("Model name :"+model)
for i, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Querying LLM"):
    text = str(row.get('text_', row.get('text', '')))  # supports both 'text_' and 'text'
    label = row.get('label', '')
    response = query_llm("Classify the sentiment of below statement into Original review or Computer-generated review. Answer in one word \n"+text)
    results.append({"text": text, "label": label, "Response": response})
output_df = pd.DataFrame(results)

# --- Save to CSV ---
output_df.to_csv("llm_output_llama3_1000.csv", index=False)
#output_df.to_csv("llm_output.csv", index=False)
#output_df.to_csv("llm_output.csv", index=False)

print("✅ Finished! Results saved as CSV file'")
display(output_df.head())

Model name :llama3.2:1b


Querying LLM: 100%|██████████| 1000/1000 [03:36<00:00,  4.61it/s]


✅ Finished! Results saved as CSV file'


,text,label,Response
0,"Love this! Well made, sturdy, and very comfor...",CG,Computer-generated review.
1,"love it, a great upgrade from the original. I...",CG,Original review.
2,This pillow saved my back. I love the look and...,CG,Original review.
3,"Missing information on how to use it, but it i...",CG,Original review
4,Very nice set. Good quality. We have had the s...,CG,Computer-generated review


In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_qwen_100.csv")

# --- Clean and standardize text ---
df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

# --- Apply your rules ---
def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

# --- Filter out UNKNOWN for accuracy ---
valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))

df.to_csv("llm_qwen_cleaned.csv", index=False)
print("\n Saved cleaned file as 'llm_qwen_cleaned.csv'")


Accuracy: 0.48

 Classification Report:
              precision    recall  f1-score   support

          CG       0.52      0.30      0.38        50
          OR       0.46      0.68      0.55        44

    accuracy                           0.48        94
   macro avg       0.49      0.49      0.47        94
weighted avg       0.49      0.48      0.46        94


 Saved cleaned file as 'llm_qwen_cleaned.csv'


In [12]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_gemma3_1000.csv")

df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))


df.to_csv("llm_gemma3_cleaned_1000.csv", index=False)
print("\n Saved cleaned file as 'llm_gemma3_cleaned_1000.csv'")


Accuracy: 0.49

 Classification Report:
              precision    recall  f1-score   support

          CG       0.70      0.04      0.08       519
          OR       0.49      0.98      0.65       481

    accuracy                           0.49      1000
   macro avg       0.59      0.51      0.37      1000
weighted avg       0.60      0.49      0.36      1000


 Saved cleaned file as 'llm_gemma3_cleaned_1000.csv'


In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# --- Load your CSV ---
df = pd.read_csv("D:\OVGU _Saurabh\SEM 3\Review Project\Code\Models\llm_output_deepseek_100.csv")

df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))

df.to_csv("llm_deepseek_cleaned.csv", index=False)
print("\n Saved cleaned file as 'llm_deepseek_cleaned.csv'")


Accuracy: 0.49

 Classification Report:
              precision    recall  f1-score   support

          CG       0.60      0.22      0.32        55
          OR       0.46      0.82      0.59        45

    accuracy                           0.49       100
   macro avg       0.53      0.52      0.46       100
weighted avg       0.54      0.49      0.44       100


 Saved cleaned file as 'llm_deepseek_cleaned.csv'


In [19]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# --- Load your CSV ---
df = pd.read_csv("/Users/kumar/fake_reviews_prediction/Models/llm_output_llama3_1000.csv")

df['label'] = df['label'].astype(str).str.strip().str.upper()
df['Response'] = df['Response'].astype(str).str.strip().str.lower()

def normalize_response(resp: str) -> str:
    if "computer" in resp:
        return "CG"
    elif "original" in resp:
        return "OR"
    else:
        return "UNKNOWN"

df['Predicted'] = df['Response'].apply(normalize_response)

valid_df = df[df['Predicted'] != "UNKNOWN"]

accuracy = accuracy_score(valid_df['label'], valid_df['Predicted'])
print(f"Accuracy: {accuracy:.2f}")

print("\n Classification Report:")
print(classification_report(valid_df['label'], valid_df['Predicted']))

df.to_csv("llm_llama3_1000_cleaned.csv", index=False)
print("\n Saved cleaned file as 'llm_llama3_1000_cleaned.csv'")


Accuracy: 0.48

 Classification Report:
              precision    recall  f1-score   support

          CG       0.51      0.09      0.15       518
          OR       0.48      0.91      0.63       481

    accuracy                           0.48       999
   macro avg       0.50      0.50      0.39       999
weighted avg       0.50      0.48      0.38       999


 Saved cleaned file as 'llm_llama3_1000_cleaned.csv'
